## Parsing data

In [46]:
# Define the path to the text file
text_file = "english_sinhala_sentences.txt"

# Open the text file and read its contents
with open(text_file, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")[:-1]

# Initialize an empty list to store the English-Sinhala pairs
text_pairs = []

# Process each line in the text file
for line in lines:
    # Split the line into English and Sinhala parts using the tab character as separator
    eng, sinhala = line.split("\t")

    # Append the English sentence and its Sinhala translation with start and end tokens
    # You can modify this part based on your specific requirements
    text_pairs.append((eng, "[start] " + sinhala + " [end]"))

# Print the first few pairs for verification
for pair in text_pairs[:5]:
    print(pair)


('I believe in the power of kindness and compassion.', '[start] කරුණාවන්තත්වයෙන් හා අනුකම්පාවේ බලය මම විශ්වාස කරමි. [end]')
("He's grateful for the love and support of his friends.", '[start] ඔහුගේ මිතුරන්ගේ ආදරය හා සහයෝගය ගැන ඔහු කෘත ful වෙනවා. [end]')
("We're inspired by the resilience of the human spirit.", '[start] මානව ආත්මයේ ඔරොත්තු දීමේ හැකියාව නිසා අපි දේවානුභාවයෙන්. [end]')
("She's determined to follow her dreams.", '[start] ඇයගේ සිහින අනුගමනය කිරීමට ඇය අධිෂ් was ාන කරගෙන සිටී. [end]')
("He's excited about the future and what it holds.", '[start] ඔහු අනාගතය ගැන සහ එය දරන දේ ගැන කලබල වේ. [end]')


In [47]:
import random

# Shuffle the text pairs randomly
random.shuffle(text_pairs)

# Calculate the number of validation and training samples
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples

# Split the text pairs into training, validation, and test sets
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

# Print the number of pairs in each set
print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")


570 total pairs
400 training pairs
85 validation pairs
85 test pairs


## Vectorizing the text data

In [48]:
import string
import re
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Define characters to be stripped from the text
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

# Define constants for vectorization
vocab_size = 15000
sequence_length = 20

# Define custom standardization function
def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(lowercase, "[%s]" % re.escape(strip_chars), "")

# Initialize TextVectorization layers for English and Sinhala
eng_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
sinhala_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,  # Add 1 for the end token
    standardize=custom_standardization,
)

# Extract English and Sinhala texts from the training pairs
train_eng_texts = [pair[0] for pair in train_pairs]
train_sinhala_texts = [pair[1] for pair in train_pairs]

# Adapt the vectorization layers to the training data
eng_vectorization.adapt(train_eng_texts)
sinhala_vectorization.adapt(train_sinhala_texts)


In [49]:
import tensorflow as tf

# Define batch_size
batch_size = 64

# Define the format_dataset function
def format_dataset(eng, sinhala):
    eng = eng_vectorization(eng)
    sinhala = sinhala_vectorization(sinhala)
    return (
        {
            "encoder_inputs": eng,
            "decoder_inputs": sinhala[:, :-1],
        },
        sinhala[:, 1:],
    )

# Define the make_dataset function
def make_dataset(pairs):
    eng_texts, sinhala_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    sinhala_texts = list(sinhala_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, sinhala_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.cache().shuffle(2048).prefetch(16)

# Create training and validation datasets
train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)


In [50]:
for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
    print(f"targets.shape: {targets.shape}")


inputs["encoder_inputs"].shape: (64, 20)
inputs["decoder_inputs"].shape: (64, 20)
targets.shape: (64, 20)


## Model building

In [51]:
import tensorflow as tf
from tensorflow.keras import layers

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        # Initialization
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        # Layers
        self.attention = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = tf.keras.Sequential([
            layers.Dense(dense_dim, activation="relu"),
            layers.Dense(embed_dim)
        ])
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):
        # Padding mask
        if mask is not None:
            padding_mask = tf.cast(mask[:, None, :], dtype="int32")
        else:
            padding_mask = None

        # Multi-head attention
        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=padding_mask
        )

        # Layer normalization and residual connection
        proj_input = self.layernorm_1(inputs + attention_output)

        # Feed-forward network
        proj_output = self.dense_proj(proj_input)

        # Layer normalization and residual connection
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "dense_dim": self.dense_dim,
            "num_heads": self.num_heads
        })
        return config


class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)
        # Initialization
        self.token_embeddings = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.position_embeddings = layers.Embedding(input_dim=sequence_length, output_dim=embed_dim)
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        # Positional encoding
        length = tf.shape(inputs)[-1]
        positions = tf.range(0, length, 1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        # Masking
        if mask is None:
            return None
        else:
            return tf.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update({
            "sequence_length": self.sequence_length,
            "vocab_size": self.vocab_size,
            "embed_dim": self.embed_dim
        })
        return config


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        # Initialization
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        # Layers
        self.attention_1 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = tf.keras.Sequential([
            layers.Dense(latent_dim, activation="relu"),
            layers.Dense(embed_dim)
        ])
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, encoder_outputs, mask=None):
        # Causal attention mask
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, None, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        else:
            padding_mask = None

        # Self-attention
        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, attention_mask=causal_mask
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        # Encoder-decoder attention
        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        # Feed-forward network
        proj_output = self.dense_proj(out_2)

        # Layer normalization and residual connection
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, None]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, sequence_length, sequence_length))
        mult = tf.concat([
            tf.expand_dims(batch_size, -1),
            tf.convert_to_tensor([1, 1])
        ], axis=0)
        return tf.tile(mask, mult)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "latent_dim": self.latent_dim,
            "num_heads": self.num_heads
        })
        return config


In [59]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define model parameters
embed_dim = 128       # 256
latent_dim = 1024     # 2048
num_heads = 4         # 8
sequence_length = 20
vocab_size = 10000    # 15000

# Define encoder input
encoder_inputs = tf.keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")

# Add positional embedding layer to encoder inputs
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)

# Apply Transformer encoder layer
encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)

# Define encoder model
encoder = tf.keras.Model(encoder_inputs, encoder_outputs)

# Define decoder input
decoder_inputs = tf.keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")

# Define encoded sequence inputs
encoded_seq_inputs = tf.keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")

# Add positional embedding layer to decoder inputs
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)

# Apply Transformer decoder layer
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, encoded_seq_inputs)

# Add dropout layer
x = layers.Dropout(0.5)(x)

# Output dense layer
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)

# Define decoder model
decoder = tf.keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

# Connect decoder to encoder outputs
decoder_outputs = decoder([decoder_inputs, encoder_outputs])

# Define the complete Transformer model
transformer = tf.keras.Model(
    [encoder_inputs, decoder_inputs], decoder_outputs, name="transformer"
)


## Model training

In [60]:
epochs = 1  # This should be at least 30 for convergence

transformer.summary()
transformer.compile(
    "rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
transformer.fit(train_ds, epochs=epochs, validation_data=val_ds)

Model: "transformer"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 encoder_inputs (InputLayer  [(None, None)]               0         []                            
 )                                                                                                
                                                                                                  
 positional_embedding_10 (P  (None, None, 128)            1282560   ['encoder_inputs[0][0]']      
 ositionalEmbedding)                                                                              
                                                                                                  
 decoder_inputs (InputLayer  [(None, None)]               0         []                            
 )                                                                                      

In [78]:
import pandas as pd
import numpy as np


## Manual testing

In [80]:
import keras.ops as ops

sinhala_vocab = sinhala_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(sinhala_vocab)), sinhala_vocab))
max_decoded_sentence_length = 20


def decode_sequence(input_sentence):
    tokenized_input_sentence = eng_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = sinhala_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])

        # ops.argmax(predictions[0, i, :]) is not a concrete value for jax here
        sampled_token_index = ops.convert_to_numpy(
            ops.argmax(predictions[0, i, :])
        ).item(0)
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break
    return decoded_sentence


test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(30):
    input_sentence = random.choice(test_eng_texts)
    translated = decode_sequence(input_sentence)

ModuleNotFoundError: No module named 'keras.ops'